# Exporting a disclosure file from lcopt

Note: For this to work you need the latest version of lcopt-dev from the `disclosures` branch on github

### Load (or create) an lcopt model

In [1]:
from lcopt import *

In [17]:
model = LcoptModel(load=r"files\Example Model.lcopt") # Note: you can download 'Example Model.lcopt' from the files folder on github

Here's what the model looks like:

![Flow chart](images/tea_flow_chart.jpg)

### Create the disclosure file and store the filename

Using `export_disclosure` with no arguments exports the unspecified model (all values 1)

(Note: `folder_path` optionally chooses a folder to put the files in, otherwise they are saved to the working directory)

In [3]:
unspecified_file_name = model.export_disclosure(folder_path='files')
unspecified_file_name

'files\\Example_Model_unspecified.json'

Or you can specify a parameter set by its index:

In [4]:
ps0_file_name = model.export_disclosure(0, folder_path='files')
ps0_file_name

'files\\Example_Model_ps_0.json'

or its name:

In [5]:
ps1_file_name = model.export_disclosure('Milky tea', folder_path='files')
ps1_file_name

'files\\Example_Model_ps_Milky tea.json'

### Load one of the files to see the format

In [6]:
import json
from pprint import pprint

In [7]:
with open(unspecified_file_name, 'r') as j:
    data = json.load(j)

In [8]:
pprint(data)

{'Ad': {'data': [[[0, 1], 1.0], [[1, 1], 1.0], [[2, 2], 1.0], [[3, 0], 1.0]],
        'shape': [4, 3]},
 'Af': {'data': [[[1, 2], 1.0], [[2, 0], 1.0]], 'shape': [3, 3]},
 'Bf': {'data': [[[0, 1], 1.0]], 'shape': [1, 3]},
 'background flows': [{'brightway_id': ['Ecoinvent3_3_cutoff',
                                        'b80c575f22df17a0fbc7b4ae11f65dd2'],
                       'ecoinvent_id': '832cb6db-89ea-45a8-878b-42a580a3e33e',
                       'ecoinvent_name': 'market for electricity, medium '
                                         'voltage',
                       'index': 0},
                      {'brightway_id': ['Ecoinvent3_3_cutoff',
                                        '05013963d77778b2c11336e1448b1fe2'],
                       'ecoinvent_id': '08852717-6b16-426c-be3d-649b2e6381b1',
                       'ecoinvent_name': 'market for tap water',
                       'index': 1},
                      {'brightway_id': ['Ecoinvent3_3_cutoff',
              

### Reconstruct the matrices

Here's what wer're expecting:

![disclosure_pic.png](images\disclosure_pic.png)

In [9]:
import numpy as np
import pandas as pd

Write a helper function to reconstruct a matrix as a numpy array from the json data

In [10]:
def reconstruct_matrix(matrix_dict):
    m = np.zeros(matrix_dict['shape'])
    for (r, c), v in matrix_dict['data']:
        m[r, c] = v
    
    return m

Create pandas DataFrames for easy viewing of the resulting matrices

In [11]:
Af = pd.DataFrame(reconstruct_matrix(data['Af']), index=["({}) {}".format(x['index'], x['name']) for x in data['foreground flows']])

In [12]:
Ad = pd.DataFrame(reconstruct_matrix(data['Ad']), index=["{} ({}, {})".format(x['ecoinvent_name'], *x['brightway_id']) for x in data['background flows']])

In [13]:
Bf = pd.DataFrame(reconstruct_matrix(data['Bf']), index=["{} ({}, {})".format(x['name'], *x['biosphere3_id']) for x in data['foreground emissions']])

In [14]:
Af

,0,1,2
(0) Tea,0.0,0.0,0.0
(1) Boiling water,0.0,0.0,1.0
(2) Black tea,1.0,0.0,0.0


In [15]:
Ad

,0,1,2
"market for electricity, medium voltage (Ecoinvent3_3_cutoff, b80c575f22df17a0fbc7b4ae11f65dd2)",0.0,1.0,0.0
"market for tap water (Ecoinvent3_3_cutoff, 05013963d77778b2c11336e1448b1fe2)",0.0,1.0,0.0
"market for tea, dried (Ecoinvent3_3_cutoff, 45996ff4d44c8a72ba5bf0ad89fbdb5e)",0.0,0.0,1.0
"market for cow milk (Ecoinvent3_3_cutoff, 214da18debd3c0074df1c1996cf0fed4)",1.0,0.0,0.0


In [16]:
Bf

,0,1,2
"Water, emission, air (biosphere3, 075e433b-4be4-448e-9510-9a5029c1ce94)",0.0,1.0,0.0


Done!